In [1]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from abelfunctions import RiemannTheta

import layers as layers
import model as mdl


In [2]:
def mse(x,y):
    return x+y

In [3]:
M = mdl.Model()
M.add(layers.ThetaUnitLayer(1,2))
M.add(layers.ThetaUnitLayer(2,1))
M.set_costfunction(mse)

In [6]:
M.get_cost(51,1)

52

In [5]:



R = np.linspace(-100, 100, 100).reshape(1,100)
#R=np.random.rand(5).reshape((1,5))


In [ ]:
O = M.feedthrough(R)


In [ ]:
plt.plot(R.flatten(), np.real(O.flatten()),"ob-")

In [ ]:
O.flatten()